# Setup

In [ ]:
EXP_NAME='Santander Release Pipeline'
OUTPUT_DIR='s3://kubeflow-pipelines-demo/tfx'

PROJECT_NAME = 'kf-pipelines'

# Create a release experiment in the Kubeflow pipeline

Kubeflow Pipeline requires having an Experiment before making a run. An experiment is a group of comparable runs

In [ ]:
import kfp
from kfp import compiler
import kfp.dsl as dsl

import kfp.notebook
import kfp.gcp as gcp

client = kfp.Client()
exp = client.create_experiment(name=EXP_NAME)

# Define the release pipeline

In [ ]:
import kfp
from kfp import components
from kfp import dsl
from kfp import gcp
from kfp import onprem
import os

platform = 'Azure'

kubeflow_deploy_op = components.load_component_from_file('pipeline_steps/serving/deployer/component.yaml')
minio_endpoint = os.environ.get('MINIO_URL', 'minio-service.kubeflow.svc.cluster.local:9000')
minio_key = os.environ.get('MINIO_KEY', 'minio')
minio_secret = os.environ.get('MINIO_SECRET', 'XXXXXX')

print('Minio parameters : URL ', minio_endpoint, ' key ', minio_key, ' secret ', minio_secret)

os.environ['AWS_ACCESS_KEY_ID'] = minio_key
os.environ['AWS_SECRET_ACCESS_KEY'] = minio_secret
os.environ['AWS_REGION'] = 'us-west-1'
os.environ['S3_REGION'] = 'us-west-1'
os.environ['S3_ENDPOINT'] = minio_endpoint
os.environ['S3_USE_HTTPS'] = '0'
os.environ['S3_VERIFY_SSL'] = '0'

In [ ]:
from kubernetes import client as k8s_client

@dsl.pipeline(
    name='Santander Customer Transaction Prediction Release Pipeline',
    description='Example pipeline that releases the trained classification model for Santander customer transaction.'
)
def santander_transaction_classification(
        output,
        project,
):
    tf_server_name = 'kfdemo-service'

    if platform != 'Azure':
        vop = dsl.VolumeOp(
            name="create_pvc",
            resource_name="pipeline-pvc",
            modes=dsl.VOLUME_MODE_RWM,
            size="1Gi"
        )

        checkout = dsl.ContainerOp(
            name="checkout",
            image="alpine/git:latest",
            command=["git", "clone", "https://github.com/kubeflow/pipelines.git", str(output) + "/pipelines"],
        ).apply(onprem.mount_pvc(vop.outputs["name"], 'local-storage', output))
        checkout.after(vop)

    if platform == 'Azure':
        deploy = kubeflow_deploy_op(
            cluster_name=project,
            model_dir=str(
                's3://kubeflow-pipelines-demo/tfx/1ea17045-2829-4d85-8459-1807988ec0fa/santander-customer-transaction-prediction-tgdfd-1263318984/data') + '/export/export',
            server_name=tf_server_name
        )
    else:
        deploy = kubeflow_deploy_op(
            cluster_name=project,
            model_dir=str(
                's3://kubeflow-pipelines-demo/tfx/1ea17045-2829-4d85-8459-1807988ec0fa/santander-customer-transaction-prediction-tgdfd-1263318984/data') + '/export/export',
            pvc_name=vop.outputs["name"],
            server_name=tf_server_name
        )

    webapp = dsl.ContainerOp(
        name='webapp',
        image='k8scc01covidacr.azurecr.io/kubeflow-webapp:v0.2',
        arguments=["--model_name", 'santanderapp']

    )
    webapp.after(deploy)

    steps = [deploy, webapp]

    for step in steps:
        if platform == 'GCP':
            step.apply(gcp.use_gcp_secret('user-gcp-sa'))
        elif platform != 'Azure':
            step.apply(onprem.mount_pvc(vop.outputs["name"], 'local-storage', output))

    dsl.get_pipeline_conf().set_image_pull_secrets([k8s_client.V1ObjectReference(name="k8scc01covidacr-registry-connection")])
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='AWS_ACCESS_KEY_ID', value=os.environ['AWS_ACCESS_KEY_ID'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='AWS_SECRET_ACCESS_KEY', value=os.environ['AWS_SECRET_ACCESS_KEY'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='AWS_REGION', value=os.environ['AWS_REGION'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='S3_REGION', value=os.environ['S3_REGION'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='S3_ENDPOINT', value=os.environ['S3_ENDPOINT'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='S3_USE_HTTPS', value=os.environ['S3_USE_HTTPS'])))
    dsl.get_pipeline_conf().add_op_transformer(lambda cop: cop.container.add_env_variable(k8s_client.V1EnvVar(name='S3_VERIFY_SSL', value=os.environ['S3_VERIFY_SSL'])))

# Compile the pipeline

Compile the pipeline into a tar package

In [ ]:
compiler.Compiler().compile(santander_transaction_classification, 'santander_release_pipeline.zip')

# Submit the run with parameters

In [ ]:
import datetime
import time

run = client.run_pipeline(
    exp.id, 
    'santander_release_pipeline-' + time.strftime("%Y%m%d-%H%M%S"), 
    'santander_release_pipeline.zip', 
    params={'output':'s3://kubeflow-pipelines-demo/tfx','project':'kf-pipelines'}
)